# PHYS-GA2059 - Statistics and Data Science for Physicists
# Student: Gehan Ranepura
## Problem Set #4 - Exercise 8 
Hogg et al. (2010). Data analysis recipes: Fitting a model to data.
https://doi.org/10.48550/arXiv.1008.4686

Compute the standard uncertainty $σ_m^2$ obtained for the slope of the line found by the standard fit you did in Exercise 2. 
Now make jackknife (20 trials) and bootstrap estimates for the uncertainty $σ_m^2$. 
How do the uncertainties compare and which seems most reasonable, given the data and uncertainties on the data?

In [1]:
import sys
import pandas as pd
import numpy as np
import ipywidgets as widgets
import IPython
import matplotlib as mpl
import matplotlib.pyplot as plt
import weighted_linear_least_square_fit as llsq

from numpy.linalg import inv
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')
from pylab import cm

''' Set up Plot Style ''' 

# plt.style.use(['science','nature'])
plt.style.use('classic')
mpl.rcParams['axes.grid'] = True
mpl.rcParams['axes.grid.which'] = 'both'
mpl.rcParams['xtick.minor.visible'] = True

## use (4,3) for 2 column plots, (8,6) for single plots.
fgsz_1 = (8,6)
fgsz_2 = (4,3)

In [2]:
data_file = np.loadtxt("data.txt", dtype=float)
data = data_file[:,0:3]
print(data)

#  Read data file at path 'file_path'. Return a np array data = array(...).
#  Read data to ignore the first 5 data points

[[201. 592.  61.]
 [244. 401.  25.]
 [ 47. 583.  38.]
 [287. 402.  15.]
 [203. 495.  21.]
 [ 58. 173.  15.]
 [210. 479.  27.]
 [202. 504.  14.]
 [198. 510.  30.]
 [158. 416.  16.]
 [165. 393.  14.]
 [201. 442.  25.]
 [157. 317.  52.]
 [131. 311.  16.]
 [166. 400.  34.]
 [160. 337.  31.]
 [186. 423.  42.]
 [125. 334.  26.]
 [218. 533.  16.]
 [146. 344.  22.]]


# JackKnife Method:

In jackknife, you make your measurement $N$ times, but each time leaving out data point $i$.

For each trial, use standard weighted linear least-square fit but leaving out data point $i$, to obtain $m_i, b_i$: 
$$ \sigma_m^2=\frac{N-1}{N} \sum_{i=1}^N \left[m_i -m\right]^2 $$
$$ \sigma_b^2=\frac{N-1}{N} \sum_{i=1}^N \left[b_i -b\right]^2 $$

where, $m, b$ is the average over all $N$ trials.
$$ m = \frac{1}{N} \sum_{i=1}^N m_i $$ 
$$ b = \frac{1}{N} \sum_{i=1}^N b_i $$

In [1]:
N = len(data)            # Number of trials = number of data points
dpID = np.arange(0,N,1)  # Data point ID, correspond each data point with a list ID number

# Create data series in pandas
m_jkls = pd.Series([],dtype=object)
b_jkls = pd.Series([],dtype=object)
m_sig_jkls = pd.Series([],dtype=object)
b_sig_jkls = pd.Series([],dtype=object)

# Perform weighted linear least-square fit over a loop, excluding the data point i each time
for i in range(N):
    dpID_i = np.delete(dpID, i)
    data_i = data[dpID_i]
    m_jkls[i], b_jkls[i], m_sig_jkls[i], b_sig_jkls[i] = llsq.weighted_linear_least_square_fit(data_i)

# Find m,b which is merely the average of all trials (N)
m_jk = np.mean(m_jkls)
b_jk = np.mean(b_jkls)

# Find uncertainty for m,b
m_sig_jk = np.std(m_jkls)*np.sqrt(N-1)  # Uncertainity of m_jk is merely the standard deviation of m_jk * sqrt(N-1)
b_sig_jk = np.std(b_jkls)*np.sqrt(N-1)  # Uncertainity of b_jk is merely the standard deviation of b_jk * sqrt(N-1)

print("m_res_jk =  %s \pm %s"%(round(m_jk,2),round(m_sig_jk,2)) )
print("b_res_jk =  %s \pm %s"%(round(b_jk,2),round(b_sig_jk,2)) )

NameError: name 'data' is not defined

# Bootstrap Method:
In bootstrap, you make your measurement $M$ times but each time randomly choose $N$ points (with replacement) from the dataset. 

For each trial, use standard weighted linear least-square fit for each selection of N points to obtain $m_j,b_j$:
$$\sigma_m^2=\frac{1}{M} \sum_{j=1}^M\left[m_j-m\right]^2$$
$$\sigma_b^2=\frac{1}{M} \sum_{j=1}^M\left[b_j-b\right]^2$$

where, $m, b$ is the average over all $M$ trials.
$$ m = \frac{1}{M} \sum_{j=1}^M m_j $$
$$ b = \frac{1}{M} \sum_{j=1}^M b_j $$

In [7]:
N = len(data)           # Number of data points
dpID = np.arange(0,N,1) # Data point ID, correspond each data point with a list ID number
M = N+2                 # Number of trials, if M is comparable to N, there probably isn't much else you can learn.

# Create data series in pandas
m_bootls = pd.Series([],dtype=object)
b_bootls = pd.Series([],dtype=object)
m_sig_bootls = pd.Series([],dtype=object)
b_sig_bootls = pd.Series([],dtype=object)

# Perform weighted linear least-square fit over a loop, with randomized N data points from dataset
rng = np.random.default_rng()          # Random number generator
for j in range(M):
    dpID_j = rng.integers(0,N, size=N) # Create list of randomized integers from 0 to N
    data_j = data[dpID_j]              # Choose data points that correspond to the list of integers from 0 to N
    m_bootls[j], b_bootls[j], m_sig_bootls[j], b_sig_bootls[j] = llsq.weighted_linear_least_square_fit(data_j)

# Find m,b which is merely the average of all trials (M)    
m_boot = np.mean(m_bootls)    
b_boot = np.mean(b_bootls)

# Find uncertainty for m,b
m_sig_boot = np.std(m_bootls)          # Uncertainity of m_boot is merely the standard deviation of m_boot
b_sig_boot = np.std(b_bootls)          # Uncertainity of b_boot is merely the standard deviation of b_boot

print( "m_res_boot =  %s \pm %s"%(round(m_boot,2),round(m_sig_boot,2)) )
print( "b_res_boot =  %s \pm %s"%(round(b_boot,2),round(b_sig_boot,2)) )

m_res_boot =  1.17 \pm 0.66
b_res_boot =  195.76 \pm 114.45


# Questions

How do the uncertainties compare and which seems most reasonable, given the data and uncertainties on the data?

~ Both the JackKnife and Boostrap methods seem to give decent estimates for the data. Jackknife always gives the same result of the measurement of m,b and their asscoiated uncertainites. However it seems that due to random nature of data picking in the Bootstrap method, the measurements we get are always changing. Therefore, there is a higher probability of getting innaccurate measurements with the bootstrap method when a single data point is chosen too mnay times, encompassing a smaller pool of the original data.